In [ ]:
from IPython.display import HTML
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException, WebDriverException
import requests
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
import random

In [ ]:
# Lấy về danh sách các proxy IP và port từ https://sslproxies.org
def get_proxies():
    url = 'https://sslproxies.org/'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    proxies = []
    count = 0
    for row in soup.find_all('tr')[1:]:
        # Chỉ lấy khoảng 20 proxies
        if(count>=20):
            break
        columns = row.find_all('td')
        # phải có ít nhất hai elements thì mới lấy
        if columns[4].text == "elite proxy" and columns[5].text == "yes":
            ip = columns[0].text
            port = columns[1].text
            proxies.append(f'{ip}:{port}')
        count +=1
    return proxies

def get_vietnamese_proxies():
    url = 'http://free-proxy.cz/en/proxylist/country/VN/all/ping/all'
    driver = webdriver.Chrome()
    driver.get(url)
    proxies = []
    proxy_table = driver.find_element(By.ID, 'proxy_list')
    count = 0
    # Tìm table có id 'proxy_list'
    rows = proxy_table.find_elements(By.TAG_NAME, 'tr')
    for row in rows:
        if(count>=10): break
        # Find all cells (columns) in the current row
        cells = row.find_elements(By.TAG_NAME, 'td')
        # Lấy ra content của cột đầu, cột hai (index 0 and 1)
        if len(cells) >= 2:
            first_column_value = cells[0].text.strip()
            second_column_value = cells[1].text.strip()
            fifth_column_value = cells[6].text.strip()
            if(fifth_column_value == "High anonymity"):
                proxies.append(f'{first_column_value}:{second_column_value}')
                print(f"First Column: {first_column_value}, Second Column: {second_column_value}")
                count+=1
    # Đóng WebDriver
    driver.quit()
    return proxies

# lấy ra danh sách các proxies
# proxy_list = get_proxies()

# lấy ra danh sách các proxies của Việt Nam
vn_proxy_list = get_vietnamese_proxies()

# Chọn một proxy ngẫu nhiên
# random_proxy = random.choice(proxy_list)
vn_random_proxy = random.choice(vn_proxy_list)
# print('Proxy: ' + random_proxy)
print('VN Proxy: ' + vn_random_proxy)

# AUTO VIEW BNS

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
# chrome_options.add_argument('--proxy-server=%s' % vn_random_proxy)
driverBNS = webdriver.Chrome(options=chrome_options, service=ChromeService(ChromeDriverManager().install()))
# driverBNS = webdriver.Chrome()
# Hàm auto scroll, refresh sau nhiều lần và click next chương bằng Selenium
def open_and_refresh_website(url, interval_seconds, number_of_chapter = 0):
      # You need to have ChromeDriver installed: https://sites.google.com/chromium.org/driver/
    driverBNS.get(url)
    count = 0
    while(count<=1):
        for _ in range(5):
            # Dùng JavaScript để từ từ cuộn xuống
            driverBNS.execute_script("window.scrollBy(0, 220);")
            time.sleep(1)
            driverBNS.execute_script("window.scrollBy(0, 220);")
            driverBNS.execute_script("window.scrollBy(0, 220);")
        driverBNS.refresh()
        time.sleep(interval_seconds)
        count+=1
    if(number_of_chapter > 40):
        driverBNS.quit()
    if(driverBNS.find_elements(By.XPATH, '//*[@id="box-directional"]/div[1]/div[2]/a')!=[]): 
        buttonNext = driverBNS.find_element(By.XPATH, '//*[@id="box-directional"]/div[1]/div[2]/a').click()
    nexturl = driverBNS.current_url
    open_and_refresh_website(nexturl, interval_seconds, number_of_chapter+1)
    
    # Close the WebDriver when the notebook cell is done
    # This is optional and depends on whether you want to keep the browser open
    # If you want to keep the browser open, omit the next line

# Set the website URL
website_urlBNS = "https://bachngocsach.vip/dich/than-kiem-luu-lac-phap-gioi-3/904/chuong-1-sao-bang/463753.html"

# Số giây cách
refresh_interval = 3

# Gọi đến function auto refresh và auto click next chương
open_and_refresh_website(website_urlBNS, refresh_interval)